In [10]:
!pip install pygame

In [11]:
import pygame 
import random

In [12]:
menu_opciones = ["Jugar", "¿Cómo jugar?", "Créditos", "Salir"]
menu_final_opciones  = ["Volver a jugar", "Salir"]
opcion_seleccionada = 0
pescados = []
tiempo_spawn_pez = 0
INTERVALO_SPAWN = 1500  # ms
PUNTAJE_OBJETIVO = 1000
y_inicial = 250   
espaciado = 80

pygame.init()

#tamaño de la ventana 
width = 800   #ancho de la pantalla
height = 650    #Alto de la pantalla
screen = pygame.display.set_mode((width,height)) #ancho de ventana
pygame.display.set_caption('Captura los peces') #nombre del juego


#imagenes
background_img = pygame.image.load("fondo.jpg") 
gota_img = pygame.image.load("gota.png").convert_alpha() 
pez_azul =  pygame.image.load("pez azul.png").convert_alpha() 
pez_rojo =  pygame.image.load("pez rojo.png").convert_alpha() 
pez_dorado =  pygame.image.load("pez dorado.png").convert_alpha() 
gato = pygame.image.load("gato.png").convert_alpha() 
fondo_menu= pygame.image.load("fondo_menu.jpg") 
gato_triste = pygame.image.load("gato triste.jpeg")
gato_feliz = pygame.image.load("gato feliz.jpg")

#Rotar
fondo_menu = pygame.transform.rotate(fondo_menu, 90)

#Tamaños 
background_img = pygame.transform.scale(background_img, (width, height)) #transformar el tamaño de la imagen a la pantalla
fondo_menu = pygame.transform.scale(fondo_menu, (width, height))
gato_triste = pygame.transform.scale(gato_triste, (width, height))
gato_feliz = pygame.transform.scale(gato_feliz, (width, height))
gota_img = pygame.transform.scale(gota_img, (50, 50)) 
pez_azul = pygame.transform.scale(pez_azul, (50, 40)) 
pez_rojo =  pygame.transform.scale(pez_rojo, (50, 40)) 
pez_dorado =  pygame.transform.scale(pez_dorado, (50, 40)) 
gato =  pygame.transform.scale(gato, (50, 50)) 

#colores
purple = (128, 0, 128)
lila = (204, 169, 221)
black = (0,0,0)
white = (255,255,255)
red = (255,0,0)
blue = (0,0,255)
golden = (255, 215, 0)
green = (0,255,0)
orange = (255,128,0)
brown = (150, 75, 0)
pink = (255,205,197)

#Gato 
x, y = 50, height-50 
vel_x = 3 
vel_y = 2 
gravedad = 0.3 
impulso_salto = -10

#enemigos
enemigo_width = 20
enemigo_height = 20
enemigos = []

#barra de vida
vida= 100

#score
score= 0
font= pygame.font.Font(None, 36)
menu_font = pygame.font.Font("RoyalBrand-Regular.otf", 60)
titulo_font = pygame.font.Font("RoyalBrand-Regular.otf", 100)

#FPS
clock = pygame.time.Clock()

# Plataformas 
plataformas = [
    pygame.Rect(2, 537, 327, 5),   # Plataforma 1
    pygame.Rect(113, 422, 362, 5),  # Plataforma 2
    pygame.Rect(213, 300, 223, 5),   # Plataforma 3
    pygame.Rect(466, 250, 286, 5),  # Plataforma 4
    pygame.Rect(216, 147, 349, 5),  # Plataforma 4
]

def mostrar_menu():
    """
    Dibuja el menú principal del juego en pantalla.
    Función:
    - Muestra el fondo del menú.
    - Muestra el título del juego con sombra para efecto 3D.
    - Muestra las opciones del menú con sombra y resaltando la opción seleccionada.
    """
    screen.blit(fondo_menu, (0, 0))  # fondo del menú
    # Título
    titulo = titulo_font.render("Captura los peces", True, lila)
    titulo_rect = titulo.get_rect(center=(width//2, 100))
    sombra = titulo_font.render("Captura los peces", True, (0, 0, 0))
    sombra_rect = sombra.get_rect(center=(width // 2 + 3, 103))
    screen.blit(sombra, sombra_rect)
    screen.blit(titulo, titulo_rect)

    for i, opcion in enumerate(menu_opciones):
        y = y_inicial + i * espaciado

    # sombra
        sombra = menu_font.render(opcion, True, (0, 0, 0))
        sombra_rect = sombra.get_rect(center=(width // 2 + 3, y + 3))
        screen.blit(sombra, sombra_rect)

    # texto principal
        color = pink if i == opcion_seleccionada else lila
        texto = menu_font.render(opcion, True, color)
        rect = texto.get_rect(center=(width // 2, y))
        screen.blit(texto, rect)


def manejar_menu(event):
     
    """
        Maneja la interacción del jugador con el menú principal.
        Función:
        - Permite moverse entre las opciones del menú con las flechas arriba/abajo.
        - Permite seleccionar una opción con Enter.
        - Retorna un valor según la opción seleccionada: 
        "jugar", "como_jugar", "creditos" o "salir".
        - Retorna None si no se ha seleccionado ninguna opción.
    """
    global opcion_seleccionada
    if event.type == pygame.KEYDOWN:
        if event.key == pygame.K_UP:
            opcion_seleccionada = (opcion_seleccionada - 1) % len(menu_opciones)
        elif event.key == pygame.K_DOWN:
            opcion_seleccionada = (opcion_seleccionada + 1) % len(menu_opciones)
        elif event.key == pygame.K_RETURN:
            opcion = menu_opciones[opcion_seleccionada]
            if opcion == "Jugar":
                return "jugar"
            elif opcion == "¿Cómo jugar?":
                return "como_jugar"
            elif opcion == "Créditos":
                return "creditos"
            elif opcion == "Salir":
                return "salir"
    return None

def mostrar_como_jugar():
    """
    Muestra la pantalla de como jugar del juego.
    """
    screen.blit(fondo_menu, (0, 0))
    screen.blit(font.render("Captura los peces y evita las gotas!", True, lila), (150, height//2 - 20))
    screen.blit(font.render("Te mueves con flechas", True, lila), (150, height//2 + 10))
    screen.blit(font.render("ENTER para volver", True, lila), (150, height//2 + 40))

        
def mostrar_creditos():
    """
    Muestra la pantalla de créditos del juego.
    """
    screen.blit(fondo_menu, (0, 0))
    screen.blit(font.render("Creado por: Nathalia Perez", True, lila), (150, 250))
    screen.blit(font.render("ENTER: volver al menú", True, lila), (150, 290))
        
PECES = {
    "azul": {
        "tiempo": 4000, #miliseg -- 4 seg
        "score": 10, #puntaje que da
        "probabilidad": 0.90 #probabilidad de aparicion
    },
    "rojo": {
        "tiempo": 3000,
        "score": 25,
        "probabilidad": 0.75
    },
    "dorado": {
        "tiempo": 1000,
        "score": 50,
        "probabilidad": 0.50
    }
}
 
 #Funciones

def elegir_tipo_pez():
    """
    Elige aleatoriamente un tipo de pez basado en probabilidades definidas en el diccionario PECES.
    Variables utilizadas:
    - PECES: diccionario donde cada tipo de pez tiene asociada una probabilidad de aparición.
    Retorna:
    - tipo: el tipo de pez seleccionado según la probabilidad.
    """
    r = random.random()
    acumulado = 0

    for tipo, datos in PECES.items():
        acumulado += datos["probabilidad"]
        if r <= acumulado:
            return tipo


def generar_pez(plataformas):
    """
    Genera un nuevo pez en una plataforma aleatoria y lo agrega a la lista de pescados.

    Parámetros:
    - plataformas: lista de rectángulos de Pygame que representan las plataformas donde pueden aparecer peces.

    Variables utilizadas:
    - pescados: lista global donde se almacenan los peces en pantalla.
    - PECES: diccionario que contiene información de cada tipo de pez (puntos, tiempo de permanencia, etc.).
    """
    if len(pescados) >= 4:
        return

    tipo = elegir_tipo_pez()
    datos = PECES[tipo]

    plataforma = random.choice(plataformas)

    x = random.randint(plataforma.left, plataforma.right - 40)
    y = plataforma.top - 30

    rect = pygame.Rect(x, y, 40, 30)
    tiempo = datos["tiempo"]

    pescados.append([rect, tipo, tiempo])

def colisionar_pescados(personaje_rect):
    """
    Verifica si el jugador ha recogido algún pez y actualiza la puntuación.

    Parámetros:
    - personaje_rect: rectángulo de Pygame que representa la posición y tamaño del jugador.

    Variables globales:
    - score: puntuación actual del jugador.

    Variables utilizadas:
    - pescados: lista de tuplas [rect, tipo, tiempo_restante].
      - rect: posición y tamaño del pez.
      - tipo: tipo/color del pez.
      - tiempo_restante: tiempo que el pez debe permanecer en pantalla (no se usa aquí).
    - PECES: diccionario que almacena información de cada tipo de pez, incluyendo puntos otorgados.
    """
     
    global score
    for pez in pescados[:]:
        rect, tipo, _ = pez
        if personaje_rect.colliderect(rect):
            score += PECES[tipo]["score"]
            pescados.remove(pez)
def actualizar_pescados(dt):
    """
    Actualiza el temporizador de cada pez y elimina los peces cuyo tiempo se ha agotado.

    Parámetros:
    - dt: tiempo transcurrido desde el último frame.

    Variables utilizadas:
    - pescados: lista de tuplas [rect, tipo, tiempo_restante].
      - rect: posición y tamaño del pez.
      - tipo: tipo/color del pez.
      - tiempo_restante: tiempo que el pez debe permanecer en pantalla.
    """
    for pez in pescados[:]:
        pez[2] -= dt
        if pez[2] <= 0:
            pescados.remove(pez)
def dibujar_pescados():
    """
    Dibuja todos los peces en pantalla según su tipo.

    Variables utilizadas:
    - pescados: lista de tuplas con información de cada pez.
      - rect: rectángulo de Pygame que indica la posición y tamaño del pez.
      - tipo: color o tipo del pez ("azul", "rojo", "dorado").
      - _: cualquier otro dato que pueda tener la tupla.
    - screen: pantalla de Pygame donde se dibujan los peces.
    - pez_azul, pez_rojo, pez_dorado: imágenes de los peces según el tipo.
    """
    for rect, tipo, _ in pescados:
        if tipo == "azul":
            screen.blit(pez_azul, (rect.x, rect.y))
        elif tipo == "rojo":
            screen.blit(pez_rojo, (rect.x, rect.y))
        elif tipo == "dorado":
            screen.blit(pez_dorado, (rect.x, rect.y))

def dibujar_score():
    """
    Dibuja la puntuación actual del jugador en la esquina superior izquierda de la pantalla.
    """
    texto = font.render(f"Score: {score}", True, black)
    screen.blit(texto, (10, 10))

def mostrar_menu_final(score_final):
    """
    Muestra la pantalla de final de juego (derrota).

    Parámetros:
    - score_final: puntuación final del jugador al terminar el juego.

    Variables globales:
    - opcion_final: índice de la opción actualmente seleccionada (si se usa).

    Función:
    - Dibuja el fondo con el gato triste.
    - Muestra el título "Fin del juego".
    - Muestra la puntuación final.
    - Dibuja las opciones del menú final (ej. "Volver a jugar", "Salir") resaltando la opción seleccionada.
    """
    global opcion_final
    screen.blit(gato_triste, (0, 0))
    titulo = titulo_font.render("Fin del juego", True, black)
    titulo_rect = titulo.get_rect(center=(width//2, 100))
    screen.blit(titulo, titulo_rect)
    score_text = font.render(f"Puntaje: {score_final}", True, purple) # Score final
    score_rect = score_text.get_rect(center=(width//2, 550))
    screen.blit(score_text, score_rect)
    # Opciones
    for i, opcion in enumerate(menu_final_opciones):
        color = lila if i == opcion_seleccionada else black
        texto = menu_font.render(opcion, True, color)
        rect = texto.get_rect(center=(width//2, 250 + i*70))
        screen.blit(texto, rect)

def mostrar_menu_victoria(score_final):
    """
    Muestra la pantalla de victoria del juego.

    Parámetros:
    - score_final: puntuación final del jugador al terminar el juego.

    Función:
    - Dibuja el fondo con el gato feliz.
    - Muestra el título "¡GANASTE!".
    - Muestra la puntuación final.
    - Dibuja las opciones del menú final (ej. "Volver a jugar", "Salir") resaltando la opción seleccionada.
    """
    screen.blit(gato_feliz, (0, 0))
    titulo = titulo_font.render("¡GANASTE!", True, pink)
    rect = titulo.get_rect(center=(width//2, 100))
    screen.blit(titulo, rect)

    score_text = font.render(f"Puntaje: {score_final}", True, purple)
    score_rect = score_text.get_rect(center=(width//2, 510))
    screen.blit(score_text, score_rect)

    for i, opcion in enumerate(menu_final_opciones):
        color = lila if i == opcion_seleccionada else black
        texto = menu_font.render(opcion, True, color)
        rect = texto.get_rect(center=(width//2, 250 + i*70))
        screen.blit(texto, rect)

def manejar_menu_final(event):

    """
    Maneja la interacción del usuario en el menú final del juego.

    Parámetros:
    - event: evento capturado por Pygame (tecla presionada).

    Variables globales que utiliza:
    - opcion_seleccionada: índice de la opción actualmente resaltada.

    Función:
    - Permite moverse entre las opciones con las flechas arriba/abajo.
    - Permite seleccionar una opción con Enter.
    - Devuelve "jugar" si se selecciona volver a jugar, "salir" si se selecciona salir.
    """

    global opcion_seleccionada
    if event.type == pygame.KEYDOWN:
        if event.key == pygame.K_UP:
            opcion_seleccionada = (opcion_seleccionada - 1) % len(menu_final_opciones)
        elif event.key == pygame.K_DOWN:
            opcion_seleccionada = (opcion_seleccionada + 1) % len(menu_final_opciones)
        elif event.key == pygame.K_RETURN:
            if menu_final_opciones[opcion_seleccionada] == "Volver a jugar":
                return "jugar"
            elif menu_final_opciones[opcion_seleccionada] == "Salir":
                return "salir"
    return None

def reiniciar_juego():
    """ 
     Reinicia todos los valores del juego a su estado inicial.
    """
    global x, y, vel_y, vida, score, enemigos, pescados, tiempo_spawn_pez
    x, y = 400, height - 50
    vel_y = 0
    vida = 100
    score = 0
    enemigos = []
    pescados = []
    tiempo_spawn_pez = 0


#LOOP principal

estado = "menu" 
running = True

while running:
    dt = clock.tick(60)  
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        # Manejar menú principal
        if estado == "menu":
            accion = manejar_menu(event)
            if accion == "jugar":
                estado = "juego"
            elif accion == "como_jugar":
                estado = "como_jugar"
            elif accion == "creditos":
                estado = "creditos"
            elif accion == "salir":
                running = False

        elif estado in ["como_jugar", "creditos"]:
            if event.type == pygame.KEYDOWN and event.key == pygame.K_RETURN:
                estado = "menu"
                opcion_seleccionada = 0            

        #menu final: Victoria o derrota
        elif estado in ["final", "victoria"]:
            accion = manejar_menu_final(event)
            if accion == "jugar":
                reiniciar_juego()
                estado = "juego"
            elif accion == "salir":
                running = False

    # Dibujar
    if estado == "menu":
        mostrar_menu()

    elif estado == "como_jugar":
        mostrar_como_jugar()

    elif estado == "creditos":
        mostrar_creditos()
           
    elif estado == "juego":
        # Movimiento del personaje
        keys = pygame.key.get_pressed()  
        if keys[pygame.K_LEFT] and x - 25 > 0: 
            x -= vel_x 
        if keys[pygame.K_RIGHT] and x + 25 < width: 
            x += vel_x
        if keys[pygame.K_SPACE] and (y >= height - 50 or en_plataforma): 
            vel_y = impulso_salto
        
        vel_y += gravedad
        y += vel_y

# Rectángulo del personaje   
        personaje_rect = pygame.Rect(x - 25, y - 25, 50, 50)

        if y >= height - 25:
            y = height - 25
            vel_y = 0
            personaje_rect.bottom = height  

        # Colisión con plataformas 
        en_plataforma = False 
        for plataforma in plataformas: 
            if personaje_rect.colliderect(plataforma):
                if vel_y > 0:
                    y = plataforma.top - 25 
                    personaje_rect.bottom = plataforma.top 
                    vel_y = 0 
                    en_plataforma = True
        if not en_plataforma and y < height - 50  : 
            vel_y += gravedad

         # PESCADOS (enemigos)
        tiempo_spawn_pez += dt
        if tiempo_spawn_pez >= INTERVALO_SPAWN:
            generar_pez(plataformas)
            tiempo_spawn_pez = 0

        actualizar_pescados(dt)
        colisionar_pescados(personaje_rect)

    # Movimiento enemigos 
        for enemigo in enemigos: 
            enemigo.y += 5 
            if enemigo.top > height: 
                enemigos.remove(enemigo)        

    # Generar enemigos     
        if len(enemigos) < 4: 
            enemigo = pygame.Rect(random.randint(0, width - enemigo_width), 0, enemigo_width, enemigo_height) 
            enemigos.append(enemigo)

    # Colisiones con enemigos 
        for enemigo in enemigos: 
            if personaje_rect.colliderect(enemigo): 
                vida -= 10 
                enemigos.remove(enemigo)

    # Dibujo en pantalla
        screen.blit(background_img, (0,0)) 
        dibujar_score()
        dibujar_pescados()

        for enemigo in enemigos: 
            screen.blit(gota_img, (enemigo.x, enemigo.y))

        screen.blit(gato, (x - 50, y - 50))

    # Barra de vida
        barra_ancho = int((vida / 100) * 200)
        color_barra = green if vida >= 80 else orange if vida >= 50 else red
        pygame.draw.rect(screen, color_barra, (10, 40, barra_ancho, 20))
        pygame.draw.rect(screen, black, (10, 40, 200, 20), 2)   

    # Plataformas
        for plataforma in plataformas:
            pygame.draw.rect(screen, black , plataforma)  

        if vida == 0:
            estado = "final"
        if score == PUNTAJE_OBJETIVO:
            estado = "victoria"
        pass
    elif estado == "final":
        mostrar_menu_final(score)
    elif estado == "victoria":
        mostrar_menu_victoria(score)

    pygame.display.update()
pygame.quit()